In [3]:
# =======================================
# ECC ElGamal Full Demo (Flexible Encoding + Encryption + Decryption for Sentences)
# =======================================

# -----------------------------
# ECC Parameters
# -----------------------------
p = 89        # Prime number defining finite field F_p
a = 22        # Curve coefficient a in y^2 = x^3 + a*x + b (mod p)
b = 28        # Curve coefficient b

# -----------------------------
# Modular square root function
# -----------------------------
def sqrt_mod(a, p):
    """Find a modular square root of 'a' modulo 'p'. Returns first x such that x^2 ≡ a mod p"""
    for x in range(p):
        if (x*x) % p == a:
            return x
    return None

# -----------------------------
# ECC Point Operations
# -----------------------------
def inv_mod(k, p):
    """Compute modular inverse of k modulo p using Fermat's little theorem"""
    return pow(k, p-2, p)

def ec_add(P, Q):
    """Add two points P and Q on the elliptic curve"""
    if P is None: return Q
    if Q is None: return P
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 == (-y2 % p):    # P + (-P) = O (point at infinity)
        return None
    if P != Q:                           # Point addition
        m = ((y2 - y1) * inv_mod(x2 - x1, p)) % p
    else:                                # Point doubling
        m = ((3*x1*x1 + a) * inv_mod(2*y1, p)) % p
    x3 = (m*m - x1 - x2) % p
    y3 = (m*(x1 - x3) - y1) % p
    return (x3, y3)

def ec_scalar_mul(k, P):
    """Multiply point P by integer k using double-and-add"""
    R = None
    Q = P
    while k > 0:
        if k & 1:
            R = ec_add(R, Q)
        Q = ec_add(Q, Q)
        k >>= 1
    return R

def ec_neg(P):
    """Compute inverse of a point P on the curve"""
    x, y = P
    return (x, (-y) % p)

# -----------------------------
# Encoding Module
# -----------------------------
def encode_letter(ch):
    """Encode a single letter into a point on the elliptic curve with offset if needed"""
    m = ord(ch.upper()) - 65   # Map A=0, B=1, ..., Z=25
    offset = 0
    while offset < 200:
        x = (m + offset) % p
        rhs = (x*x*x + a*x + b) % p
        y = sqrt_mod(rhs, p)
        if y is not None:
            if offset == 0:
                print(f"The letter {ch} corresponds to {m}. Point ({x},{y}) belongs to E. Offset={offset}")
            else:
                skipped = ", ".join(str(m + i) for i in range(offset))
                print(f"The letter {ch} corresponds to {m}. No points for x={skipped}. First valid: ({x},{y}), Offset={offset}")
            return (x, y), offset
        offset += 1
    raise ValueError(f"Encoding failed for letter {ch}")

def encode_sentence(sentence):
    """
    Encode a full sentence:
    - Letters → ECC points with offset
    - Non-letters (spaces, punctuation, numbers) are kept as-is
    Returns a list of (point, offset) for letters, or (char, None) for non-letters
    """
    encoded = []
    for ch in sentence:
        if ch.isalpha():
            Pm, off = encode_letter(ch)
            encoded.append((Pm, off))
        else:
            encoded.append((ch, None))  # Keep non-letter characters
    return encoded

# -----------------------------
# Encryption Module
# -----------------------------
def encrypt_sentence(sentence, R, eB, k):
    """Encrypt a sentence using ECC ElGamal"""
    encoded = encode_sentence(sentence)
    r = ec_scalar_mul(k, R)
    kQ = ec_scalar_mul(k, eB)

    cipher = []
    for item in encoded:
        Pm, off = item
        if off is not None:
            C2 = ec_add(Pm, kQ)
            cipher.append((C2, off))
        else:
            cipher.append((Pm, None))  # Keep spaces/punctuation unchanged
    return r, cipher

# -----------------------------
# Decryption Module
# -----------------------------
def decode_point_with_offset(x, offset):
    """Convert a curve point x-coordinate and offset back to a letter"""
    num = (x - offset) % p
    return chr((num % 26) + 65)

def decrypt_sentence(r, cipher_with_offsets, dB):
    """Decrypt a sentence using ECC ElGamal"""
    S = ec_scalar_mul(dB, r)
    decrypted_message = ""
    for item in cipher_with_offsets:
        C2, offset = item
        if offset is not None:
            Pm = ec_add(C2, ec_neg(S))
            letter = decode_point_with_offset(Pm[0], offset)
            decrypted_message += letter
        else:
            decrypted_message += C2  # Preserve spaces/punctuation
    return decrypted_message

# -----------------------------
# Example Usage
# -----------------------------
# ECC keys
R = (17, 8)      # Base point
eB = (3, 78)     # Bob's public key
dB = 41          # Bob's private key

sentence = "HELLO WORLD!"
k = 23           # Sender's ephemeral key

# Encrypt
r, cipher = encrypt_sentence(sentence, R, eB, k)
print("\nEncrypted ciphertext (points with offsets or characters):")
for c in cipher:
    print(c)

print("\nr =", r)

# Decrypt
decrypted = decrypt_sentence(r, cipher, dB)
print("\nDecrypted sentence:", decrypted)


The letter H corresponds to 7. Point (7,13) belongs to E. Offset=0
The letter E corresponds to 4. Point (4,25) belongs to E. Offset=0
The letter L corresponds to 11. Point (11,34) belongs to E. Offset=0
The letter L corresponds to 11. Point (11,34) belongs to E. Offset=0
The letter O corresponds to 14. No points for x=14. First valid: (15,23), Offset=1
The letter W corresponds to 22. No points for x=22, 23, 24. First valid: (25,19), Offset=3
The letter O corresponds to 14. No points for x=14. First valid: (15,23), Offset=1
The letter R corresponds to 17. Point (17,8) belongs to E. Offset=0
The letter L corresponds to 11. Point (11,34) belongs to E. Offset=0
The letter D corresponds to 3. Point (3,11) belongs to E. Offset=0

Encrypted ciphertext (points with offsets or characters):
((17, 8), 0)
((60, 30), 0)
((75, 25), 0)
((75, 25), 0)
((75, 64), 1)
(' ', None)
((57, 38), 3)
((75, 64), 1)
((15, 23), 0)
((75, 25), 0)
((67, 33), 0)
('!', None)

r = (36, 22)

Decrypted sentence: HELLO WORL